In [ ]:
# if on google collab
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/Lol/models')
import c2v
import winner

data_path = '/content/drive/My Drive/Lol/data'
model_path = '/content/drive/My Drive/Lol/models'
c2v_name = 'c2v5'
experiment_name = 'winner_with_both_path2'

In [ ]:
experiment = winner.Winner(
     experiment_name,
     model_path,
     c2v_name,
     data_path
)

In [ ]:
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers import BatchNormalization, LeakyReLU, Input,ReLU, Concatenate
from keras.optimizers import Adamax,SGD
from keras.models import Sequential, Model
import tensorflow as tf

In [ ]:
if True:
  input_blue = Input(shape = (experiment.N_champs,), name='input_blue')
  B = experiment.encoderBlue(input_blue)
  B = tf.keras.layers.GaussianNoise(0.1, name='2')(B)

  input_red = Input(shape = (experiment.N_champs,), name='input_red')
  R = experiment.encoderRed(input_red)
  R = tf.keras.layers.GaussianNoise(0.1, name='4')(R)

  encoded_path = Concatenate()([B, R])
  encoded_path = Dense(40, activation='tanh')(encoded_path)
  encoded_path = BatchNormalization()(encoded_path)
  encoded_path = Dropout(0.3)(encoded_path)
  encoded_path = Dense(20, activation='tanh')(encoded_path)
  encoded_path = BatchNormalization()(encoded_path)
  encoded_path = Dropout(0.3)(encoded_path)

  full_path = Concatenate()([input_blue, input_red])
  full_path = Dense(160, activation='tanh')(full_path)
  full_path = BatchNormalization()(full_path)
  full_path = Dropout(0.3)(full_path)
  full_path = Dense(80, activation='tanh')(full_path)
  full_path = BatchNormalization()(full_path)
  full_path = Dropout(0.3)(full_path)
  full_path = Dense(20, activation='tanh')(full_path)
  full_path = BatchNormalization()(full_path)
  full_path = Dropout(0.3)(full_path)


  c = Concatenate()([encoded_path, full_path])
  c = Dense(20, activation='tanh')(c)
  c = BatchNormalization()(c)
  c = Dropout(0.3)(c)
  c = Dense(20, activation='tanh')(c)
  c = BatchNormalization()(c)
  c = Dropout(0.3)(c)
  c = Dense(1, activation='sigmoid')(c)

  model = Model([input_blue, input_red], c)

In [ ]:
if False:
  input_blue = Input(shape = (experiment.N_champs,), name='input_blue')
  B = experiment.encoderBlue(input_blue)
  B = tf.keras.layers.GaussianNoise(0.1, name='2')(B)

  input_red = Input(shape = (experiment.N_champs,), name='input_red')
  R = experiment.encoderRed(input_red)
  R = tf.keras.layers.GaussianNoise(0.1, name='4')(R)

  c = Concatenate()([B, R])
  c = Dense(80, activation='tanh')(c)
  c = BatchNormalization()(c)
  c = Dropout(0.3)(c)
  c = Dense(20, activation='tanh')(c)
  c = BatchNormalization()(c)
  c = Dropout(0.3)(c)
  c = Dense(10, activation='tanh')(c)
  c = BatchNormalization()(c)
  c = Dropout(0.3)(c)
  c = Dense(10, activation='tanh')(c)
  c = BatchNormalization()(c)
  c = Dropout(0.3)(c)
  c = Dense(1, activation='sigmoid')(c)

  model = Model([input_blue, input_red], c)

In [ ]:
if False:
  input_blue = Input(shape = (experiment.N_champs,), name='input_blue')

  input_red = Input(shape = (experiment.N_champs,), name='input_red')

  c = Concatenate()([input_blue, input_red])
  c = Dense(80, activation='tanh')(c)
  c = BatchNormalization()(c)
  c = Dropout(0.3)(c)
  c = Dense(40, activation='tanh')(c)
  c = BatchNormalization()(c)
  c = Dropout(0.3)(c)
  c = Dense(20, activation='tanh')(c)
  c = BatchNormalization()(c)
  c = Dropout(0.3)(c)
  c = Dense(10, activation='tanh')(c)
  c = BatchNormalization()(c)
  c = Dropout(0.3)(c)
  c = Dense(1, activation='sigmoid')(c)

  model = Model([input_blue, input_red], c)

In [ ]:
lr = 0.00006 #0.00008
adamax = Adamax(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
loss = tf.keras.losses.BinaryCrossentropy(
    from_logits=False, label_smoothing=0.35, reduction="auto", name="binary_crossentropy"
)

model.compile(optimizer=adamax, loss=loss, metrics=['accuracy'])

In [ ]:
experiment.add_model(model)

In [ ]:
experiment.train(1200, batch_size=512)